In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


dataset:
- 0 : negative
- 1 : neutral
- 2 : positive

# Dataset laptop review

In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize

In [ ]:
# For data train
df = pd.read_csv('drive/MyDrive/NLP/Laptop_review.csv')
df.head()

,id,Sentence,Aspect Term,polarity,from,to
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,41,45
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,74,86
2,1316,The tech guy then said the service center does...,service center,negative,27,41
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,109,121
4,1316,The tech guy then said the service center does...,tech guy,neutral,4,12


In [ ]:
df = df.drop(labels=["from", "to"], axis=1)
df = df.loc[(df["polarity"] != "conflict")]
df.head()

,id,Sentence,Aspect Term,polarity
0,2339,I charge it at night and skip taking the cord ...,cord,neutral
1,2339,I charge it at night and skip taking the cord ...,battery life,positive
2,1316,The tech guy then said the service center does...,service center,negative
3,1316,The tech guy then said the service center does...,"""sales"" team",negative
4,1316,The tech guy then said the service center does...,tech guy,neutral


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokenizer=ToktokTokenizer()
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
def tokenize_sentence(text):
  tokens = word_tokenize(text)
  return tokens

df['tokens_aspect']=df['Aspect Term'].apply(tokenize_sentence)
df['tokens']=df['Sentence'].apply(tokenize_sentence)

df.head()

,id,Sentence,Aspect Term,polarity,tokens_aspect,tokens
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,[cord],"[I, charge, it, at, night, and, skip, taking, ..."
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,"[battery, life]","[I, charge, it, at, night, and, skip, taking, ..."
2,1316,The tech guy then said the service center does...,service center,negative,"[service, center]","[The, tech, guy, then, said, the, service, cen..."
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,"[``, sales, '', team]","[The, tech, guy, then, said, the, service, cen..."
4,1316,The tech guy then said the service center does...,tech guy,neutral,"[tech, guy]","[The, tech, guy, then, said, the, service, cen..."


In [ ]:
def tags_sentence(index, arrToken, arrAspect):
  hasil = []
  i = 0
  while i < len(arrToken):
    if (arrToken[i:i+len(arrAspect)] == arrAspect):
      for j in range(1, len(arrAspect)+1):
        hasil.append(j)
        i += 1
    else:
      hasil.append(0)
      i += 1
    
    
  return hasil

hasil2 =[]
arrToken = df["tokens"].to_numpy()
arrAspect = df["tokens_aspect"].to_numpy()
for i in range(len(df)):
  hasil2.append(tags_sentence(i, arrToken[i], arrAspect[i]))

df["tags"] = hasil2

df.head()

,id,Sentence,Aspect Term,polarity,tokens_aspect,tokens,tags
0,2339,I charge it at night and skip taking the cord ...,cord,neutral,[cord],"[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2339,I charge it at night and skip taking the cord ...,battery life,positive,"[battery, life]","[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1316,The tech guy then said the service center does...,service center,negative,"[service, center]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
3,1316,The tech guy then said the service center does...,"""sales"" team",negative,"[``, sales, '', team]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1316,The tech guy then said the service center does...,tech guy,neutral,"[tech, guy]","[The, tech, guy, then, said, the, service, cen...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
def convert_polarity(polarity):
  try :
    if(polarity == "negative"):
      return 0
    elif(polarity == "neutral"):
      return 1
    elif(polarity == "positive"):
      return 2
    else : 
      raise 

  except :
    return -1

df["polarity"] = df["polarity"].apply(convert_polarity)

df.head()

,id,Sentence,Aspect Term,polarity,tokens_aspect,tokens,tags
0,2339,I charge it at night and skip taking the cord ...,cord,1,[cord],"[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,2339,I charge it at night and skip taking the cord ...,battery life,2,"[battery, life]","[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1316,The tech guy then said the service center does...,service center,0,"[service, center]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
3,1316,The tech guy then said the service center does...,"""sales"" team",0,"[``, sales, '', team]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1316,The tech guy then said the service center does...,tech guy,1,"[tech, guy]","[The, tech, guy, then, said, the, service, cen...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
def pols_sentence(index, arr, pol):
  hasil = []
  for i in (arr):
    if(i == 0):
      hasil.append(-1)
    else:
      hasil.append(pol)
  return hasil

arrTag = df["tags"].to_numpy()
pols = df["polarity"].to_numpy()
hasil2 =[]
for i in range(len(df)):
  hasil2.append(pols_sentence(i, arrTag[i], pols[i]))

df["pol"] = hasil2

df.head()

,id,Sentence,Aspect Term,polarity,tokens_aspect,tokens,tags,pol
0,2339,I charge it at night and skip taking the cord ...,cord,1,[cord],"[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1..."
1,2339,I charge it at night and skip taking the cord ...,battery life,2,"[battery, life]","[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,1316,The tech guy then said the service center does...,service center,0,"[service, center]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1,..."
3,1316,The tech guy then said the service center does...,"""sales"" team",0,"[``, sales, '', team]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,1316,The tech guy then said the service center does...,tech guy,1,"[tech, guy]","[The, tech, guy, then, said, the, service, cen...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."


In [ ]:
arrID = df["id"].to_numpy()
arrID = list(dict.fromkeys(arrID))

for id in arrID:
  dfId = df.loc[(df["id"] == id)]
  tag = dfId.tags.to_numpy()
  if len(dfId)> 1:
    for i in range(len(tag)-1):
      for j in range(len(tag[i])):
        if tag[i][j] != tag[i+1][j]:
          if tag[i][j] != 0:
            tag[i+1][j] = tag[i][j]
          else :
            tag[i][j] = tag[i+1][j]

  df.loc[(df["id"] == id)]["tags"] = tag

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,id,Sentence,Aspect Term,polarity,tokens_aspect,tokens,tags,pol
0,2339,I charge it at night and skip taking the cord ...,cord,1,[cord],"[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1..."
1,2339,I charge it at night and skip taking the cord ...,battery life,2,"[battery, life]","[I, charge, it, at, night, and, skip, taking, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,1316,The tech guy then said the service center does...,service center,0,"[service, center]","[The, tech, guy, then, said, the, service, cen...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1,..."
3,1316,The tech guy then said the service center does...,"""sales"" team",0,"[``, sales, '', team]","[The, tech, guy, then, said, the, service, cen...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,1316,The tech guy then said the service center does...,tech guy,1,"[tech, guy]","[The, tech, guy, then, said, the, service, cen...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."


In [ ]:
sent = []
for s in df['tokens']:
  temp = [str(i) for i in s]
  sent.append(str(temp))

df['tokens'] = sent

df.head()

,id,Sentence,Aspect Term,polarity,tokens_aspect,tokens,tags,pol
0,2339,I charge it at night and skip taking the cord ...,cord,1,[cord],"['I', 'charge', 'it', 'at', 'night', 'and', 's...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1..."
1,2339,I charge it at night and skip taking the cord ...,battery life,2,"[battery, life]","['I', 'charge', 'it', 'at', 'night', 'and', 's...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,1316,The tech guy then said the service center does...,service center,0,"[service, center]","['The', 'tech', 'guy', 'then', 'said', 'the', ...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1,..."
3,1316,The tech guy then said the service center does...,"""sales"" team",0,"[``, sales, '', team]","['The', 'tech', 'guy', 'then', 'said', 'the', ...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,1316,The tech guy then said the service center does...,tech guy,1,"[tech, guy]","['The', 'tech', 'guy', 'then', 'said', 'the', ...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."


In [ ]:
#df["tokens"] = [str(x) for x in df["tokens"]]
df["tokens"][0]

"['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.']"

In [ ]:
df["tags"] = [str(x) for x in df["tags"]]
df["tags"][0]

'[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0]'

In [ ]:
df["pol"] = [str(x) for x in df["pol"]]
df["pol"][0]

'[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1]'

In [ ]:
df = df.drop(labels=["id", "Sentence", "Aspect Term", "polarity", "tokens_aspect"], axis=1)
df.head()

,tokens,tags,pol
0,"['I', 'charge', 'it', 'at', 'night', 'and', 's...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1..."
1,"['I', 'charge', 'it', 'at', 'night', 'and', 's...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,"['The', 'tech', 'guy', 'then', 'said', 'the', ...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1,..."
3,"['The', 'tech', 'guy', 'then', 'said', 'the', ...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,"['The', 'tech', 'guy', 'then', 'said', 'the', ...","[0, 1, 2, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ...","[-1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1,..."


In [ ]:
df_train = df.loc[: 2000]
df_test= df.loc[2000:]

df.to_csv('laptop.csv')
df_train.to_csv('drive/MyDrive/NLP/laptop_train.csv')
df_test.to_csv('drive/MyDrive/NLP/laptop_test.csv')


# With Bert

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
import torch

In [ ]:
class dataset_ABSA(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, tags, pols = self.df.iloc[idx, :3].values
        tokens = tokens.replace("'", "").strip("][").split(', ')
        tags = tags.strip('][').split(', ')
        pols = pols.strip('][').split(', ')

        bert_tokens = []
        bert_att = []
        pols_label = 0
        for i in range(len(tokens)):
            t = self.tokenizer.tokenize(tokens[i])
            bert_tokens += t
            if int(pols[i]) != -1:
                bert_att += t
                pols_label = int(pols[i])

        segment_tensor = [0] + [0]*len(bert_tokens) + [0] + [1]*len(bert_att)
        bert_tokens = ['[cls]'] + bert_tokens + ['[sep]'] + bert_att
        

        bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)

        ids_tensor = torch.tensor(bert_ids)
        pols_tensor = torch.tensor(pols_label)
        segment_tensor = torch.tensor(segment_tensor)

        return bert_tokens, ids_tensor, segment_tensor, pols_tensor

    def __len__(self):
        return len(self.df)

## Model Bert

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.2 MB/s 
     |████████████████████████████████| 7.6 MB 35.6 MB/s 
     |████████████████████████████████| 182 kB 63.8 MB/s 


In [3]:
from transformers import BertModel
import torch

class bert_ABSA(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(bert_ABSA, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 3)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, ids_tensors, lable_tensors, masks_tensors, segments_tensors):
        _, pooled_outputs = self.bert(input_ids=ids_tensors, attention_mask=masks_tensors, token_type_ids=segments_tensors, return_dict=False)
        linear_outputs = self.linear(pooled_outputs)

        if lable_tensors is not None:
            loss = self.loss_fn(linear_outputs, lable_tensors)
            return loss
        else:
            return linear_outputs

In [14]:
class bert_ATE(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(bert_ATE, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 3)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, ids_tensors, tags_tensors, masks_tensors):
        bert_outputs,_ = self.bert(input_ids=ids_tensors, attention_mask=masks_tensors, return_dict=False)
        # print(bert_outputs.size())
        linear_outputs = self.linear(bert_outputs)
        # print(linear_outputs.size())

        if tags_tensors is not None:
            tags_tensors = tags_tensors.view(-1)
            linear_outputs = linear_outputs.view(-1,3)
            # print(linear_outputs.size())
            # print(tags_tensors.size())
            loss = self.loss_fn(linear_outputs, tags_tensors)
            return loss
        else:
            return linear_outputs

In [4]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [15]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrain_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)
model_ABSA = bert_ABSA(pretrain_model_name).to(DEVICE)
optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predicti

In [9]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model
    
def save_model(model, name):
    torch.save(model.state_dict(), name)

In [ ]:
X_train_ds = dataset_ABSA(df_train, tokenizer)
X_test_ds = dataset_ABSA(df_test, tokenizer)

In [ ]:
w,x,y,z = X_train_ds.__getitem__(0)
print(w)
print(len(w))
print(x)
print(len(x))
print(y)
print(len(y))
print(z)

['[cls]', 'i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', '.', '[sep]', 'cord']
22
tensor([  100,  1045,  3715,  2009,  2012,  2305,  1998, 13558,  2635,  1996,
        11601,  2007,  2033,  2138,  1997,  1996,  2204,  6046,  2166,  1012,
          100, 11601])
22
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
22
tensor(1)


In [ ]:
def create_mini_batch2(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [ ]:
train_loader = DataLoader(X_train_ds, batch_size=20, collate_fn=create_mini_batch2, shuffle = True)
test_loader = DataLoader(X_test_ds, batch_size=30, collate_fn=create_mini_batch2, shuffle = True)

In [ ]:
print(train_loader)

In [ ]:
def train_model_ABSA(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        correct_predictions = 0
        
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors )
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch, " batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)         

        save_model(model_ABSA, 'bert_ABSA2.pkl')
        save_model(model_ABSA, 'drive/MyDrive/NLP/bert_ABSA2.pkl')
        
def test_model_ABSA(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            trueth += list([int(i) for i in label_ids])

    return trueth, pred

In [ ]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)
    
    return word_pieces, predictions, outputs

In [ ]:
%time train_model_ABSA(train_loader, 6)

epoch: 0  batch: 1 / 98  loss: 1.1043999195098877  hr: 4  min: 12  sec: 45
epoch: 0  batch: 2 / 98  loss: 1.0888811349868774  hr: 2  min: 52  sec: 35
epoch: 0  batch: 3 / 98  loss: 1.1080188751220703  hr: 2  min: 41  sec: 38
epoch: 0  batch: 4 / 98  loss: 1.0869856476783752  hr: 2  min: 32  sec: 42
epoch: 0  batch: 5 / 98  loss: 1.0846914052963257  hr: 2  min: 29  sec: 29
epoch: 0  batch: 6 / 98  loss: 1.0872076551119487  hr: 2  min: 32  sec: 7
epoch: 0  batch: 7 / 98  loss: 1.0659072228840418  hr: 2  min: 28  sec: 31
epoch: 0  batch: 8 / 98  loss: 1.0670467764139175  hr: 2  min: 32  sec: 28
epoch: 0  batch: 9 / 98  loss: 1.0648455487357245  hr: 2  min: 31  sec: 41
epoch: 0  batch: 10 / 98  loss: 1.0596205115318298  hr: 2  min: 31  sec: 4
epoch: 0  batch: 11 / 98  loss: 1.0551201646978206  hr: 2  min: 28  sec: 34
epoch: 0  batch: 12 / 98  loss: 1.0503843873739243  hr: 2  min: 30  sec: 16
epoch: 0  batch: 13 / 98  loss: 1.0509944833241975  hr: 2  min: 29  sec: 54
epoch: 0  batch: 14 / 9

## Load Model

In [ ]:
model_ABSA = load_model(model_ABSA, 'drive/MyDrive/NLP/bert_ABSA2.pkl')

## Test Model

In [ ]:
# test on train dataset
%time x, y = test_model_ABSA(train_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)], zero_division = 0))

CPU times: user 6min 3s, sys: 26.6 s, total: 6min 30s
Wall time: 6min 31s
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       731
           1       0.98      0.98      0.98       384
           2       1.00      0.98      0.99       842

    accuracy                           0.99      1957
   macro avg       0.99      0.99      0.99      1957
weighted avg       0.99      0.99      0.99      1957



In [ ]:
# test on test dataset
%time x, y = test_model_ABSA(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)], zero_division = 0))

CPU times: user 1min 20s, sys: 3.48 s, total: 1min 24s
Wall time: 1min 29s
              precision    recall  f1-score   support

           0       0.71      0.89      0.79       150
           1       0.62      0.54      0.58        71
           2       0.90      0.72      0.80       136

    accuracy                           0.75       357
   macro avg       0.74      0.71      0.72       357
weighted avg       0.77      0.75      0.75       357



## ABSA Test

In [ ]:
text = "For the price you pay this product is very good. However, battery life is a little lack-luster coming from a MacBook Pro."
aspect = ["price", "battery life"]
print(predict_model_ABSA(text, "battery life", tokenizer))

(['[cls]', 'for', 'the', 'price', 'you', 'pay', 'this', 'product', 'is', 'very', 'good', '.', 'however', ',', 'battery', 'life', 'is', 'a', 'little', 'lack', '-', 'lust', '##er', 'coming', 'from', 'a', 'mac', '##book', 'pro', '.', '[sep]', 'battery', 'life'], tensor([0]), tensor([[ 2.5918, -2.4663, -0.4769]]))


In [ ]:
text = "The graphics are stunning"
print(predict_model_ABSA(text, "graphics", tokenizer))

(['[cls]', 'the', 'graphics', 'are', 'stunning', '[sep]', 'graphics'], tensor([2]), tensor([[-2.1483, -2.4717,  4.4044]]))


# Main ABSA

In [16]:
model_ABSA = load_model(model_ABSA, 'drive/MyDrive/NLP/bert_ABSA2.pkl')
model_ATE = load_model(model_ATE, 'drive/MyDrive/NLP/bert_ATE.pkl')

In [11]:
def conv_pol(pol):
  if pol == 0 :
    return "Negative"
  elif pol == 1:
    return "Neutral"
  elif pol == 2:
    return "Positive"
  else:
    return "None"

In [17]:
def predict_model_ABSA(sentence, aspect, tokenizer):
    t1 = tokenizer.tokenize(sentence)
    t2 = tokenizer.tokenize(aspect)

    word_pieces = ['[cls]']
    word_pieces += t1
    word_pieces += ['[sep]']
    word_pieces += t2

    segment_tensor = [0] + [0]*len(t1) + [0] + [1]*len(t2)

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    segment_tensor = torch.tensor(segment_tensor).to(DEVICE)

    with torch.no_grad():
        outputs = model_ABSA(input_tensor, None, None, segments_tensors=segment_tensor)
        _, predictions = torch.max(outputs, dim=1)
    
    return word_pieces, predictions, outputs

def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()

    return word_pieces, predictions, outputs

def Main_ABSA(text):
    terms = []
    word = ""
    x, y, z = predict_model_ATE(text, tokenizer)
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace(" ##",""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])
            
    
    if len(word) != 0:
            terms.append(word.replace(" ##",""))
            
    print("Aspect :", terms)
    
    print("Sentiment :")
    if len(terms) != 0:
        for i in terms:
            _, c, p = predict_model_ABSA(text, i, tokenizer)
            print("Aspect term:", [i], "Class:", conv_pol(int(c)))


In [18]:
text = "For the price you pay this product is very good. However, battery life is a little lack-luster coming from a MacBook Pro."
Main_ABSA(text)

Aspect : ['battery life']
Sentiment :
Aspect term: ['battery life'] Class: Negative


In [19]:
text2 = "It absolutely is more expensive than most PC laptops, but the ease of use, security, and minimal problems that have arisen make it well worth the pricetag."
Main_ABSA(text2)

Aspect : ['use', 'security', ',', 'it']
Sentiment :
Aspect term: ['use'] Class: Positive
Aspect term: ['security'] Class: Positive
Aspect term: [','] Class: Negative
Aspect term: ['it'] Class: Positive


In [20]:
text3 = "The graphics are stunning"
Main_ABSA(text3)

Aspect : ['graphics']
Sentiment :
Aspect term: ['graphics'] Class: Positive
